In [2]:
import numpy as np 
import torch

In [14]:
grad_states = np.load('./images/test_training/runs339/grad_states.npy', allow_pickle=True)
print(f'grad_states.len: {len(grad_states)}')
step_size = 1/pow(2, 13)
ceil = pow(2, 0)
for bi in range(len(grad_states)):
    if bi == 0:
        print(f'param.names.len: {grad_states[bi].keys()}')

    gmin = 1e10 if bi % 40 ==0 else gmin
    cnt_higherer_ceil = 0 if bi % 40 ==0 else cnt_higherer_ceil
    cnt_lower_stepsize = 0 if bi % 40 ==0 else cnt_lower_stepsize
    cnt_total = 0 if bi % 40 ==0 else cnt_total
    for key, grad in grad_states[bi].items():
        grad_abs = abs(grad).reshape(-1)
        step_size_ = step_size * np.ones(grad_abs.shape[0])

        cnt_total += grad_abs.shape[0]
        cnt_higherer_ceil += sum(grad_abs > ceil)
        cnt_lower_stepsize += sum(grad_abs <= step_size_)
        gmin = min(gmin, abs(grad).min())
    
    if bi % 40 == 0:
        print(f'bi: {bi}, gmin: {gmin}')
        print(f'bi: {bi}, {(100*cnt_higherer_ceil)/cnt_total:.2f}% higher than the ceil, {(100*cnt_lower_stepsize)/cnt_total:.2f}% lower than the step_size')

grad_states.len: 400
param.names.len: dict_keys(['features.0.0.weight', 'features.0.1.weight', 'features.0.1.bias', 'features.1.0.block.0.0.weight', 'features.1.0.block.0.1.weight', 'features.1.0.block.0.1.bias', 'features.1.0.block.1.fc1.weight', 'features.1.0.block.1.fc1.bias', 'features.1.0.block.1.fc2.weight', 'features.1.0.block.1.fc2.bias', 'features.1.0.block.2.0.weight', 'features.1.0.block.2.1.weight', 'features.1.0.block.2.1.bias', 'features.2.0.block.0.0.weight', 'features.2.0.block.0.1.weight', 'features.2.0.block.0.1.bias', 'features.2.0.block.1.0.weight', 'features.2.0.block.1.1.weight', 'features.2.0.block.1.1.bias', 'features.2.0.block.2.fc1.weight', 'features.2.0.block.2.fc1.bias', 'features.2.0.block.2.fc2.weight', 'features.2.0.block.2.fc2.bias', 'features.2.0.block.3.0.weight', 'features.2.0.block.3.1.weight', 'features.2.0.block.3.1.bias', 'features.2.1.block.0.0.weight', 'features.2.1.block.0.1.weight', 'features.2.1.block.0.1.bias', 'features.2.1.block.1.0.weight

In [11]:
n_layers = 10
step_size = 1/pow(2, 14)
for bi in range(len(grad_states)):
    if bi > 40 :
        break
    states = {}
    layer_pre = 0
    cnt, cnt_total = 0, 0
    for name, grad in grad_states[bi].items():
        feature = name.split('.')[0]
        layer = int(name.split('.')[1])
        if feature == 'classifier':
            layer = 9
        if layer != layer_pre:
            print(f'layer: {layer_pre}, {(100*cnt)/cnt_total:.4f}% lower than the step_size')
            cnt, cnt_total = 0, 0
            layer_pre = layer

        type = '.'.join(name.split('.')[-2:])
        if type == '0.weight':
            grad_abs = abs(grad).reshape(-1)
            step_size_ = step_size * np.ones(grad_abs.shape[0])
            cnt_total += grad_abs.shape[0]
            cnt += sum(grad_abs <= step_size_)


layer: 0, 4.7454% lower than the step_size
layer: 1, 1.6250% lower than the step_size
layer: 2, 4.0892% lower than the step_size
layer: 3, 7.5126% lower than the step_size
layer: 4, 12.9037% lower than the step_size
layer: 5, 13.2162% lower than the step_size
layer: 6, 17.8720% lower than the step_size
layer: 7, 13.0893% lower than the step_size
layer: 8, 11.8569% lower than the step_size
layer: 0, 6.2500% lower than the step_size
layer: 1, 2.3750% lower than the step_size
layer: 2, 3.8724% lower than the step_size
layer: 3, 7.8178% lower than the step_size
layer: 4, 13.5963% lower than the step_size
layer: 5, 13.3309% lower than the step_size
layer: 6, 18.1593% lower than the step_size
layer: 7, 13.2643% lower than the step_size
layer: 8, 12.1782% lower than the step_size
layer: 0, 5.2083% lower than the step_size
layer: 1, 1.3750% lower than the step_size
layer: 2, 4.4145% lower than the step_size
layer: 3, 7.7020% lower than the step_size
layer: 4, 13.2241% lower than the step_size


KeyboardInterrupt: 